In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import nltk
import ast
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import joblib
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [3]:
df = pd.read_csv(r'df_modificado.csv')

decido hacer el sistema de recomendacion con las columnas que poseen texto por lo que elimino las columnas numericas.

In [5]:
df = df.drop(['crew', 'popularity','runtime','vote_average','vote_count'], axis=1)

De la columna cast tomo los primeros 3 actores y aplico modificaciones para quitar corchetes y comillas.

In [6]:
df['cast'] = df['cast'].apply(lambda x: x.split(",")[:3] if isinstance(x, str) else x)

In [7]:
df['cast'] = df['cast'].astype(str).str.replace('[', '').str.replace(']', '')

In [8]:
df['cast'] = df['cast'].str.replace("'", "")

In [9]:
primer_registro = df['cast'].iloc[0]
print(primer_registro)

Tom Hanks,  Tim Allen,  Don Rickles


Creo una columna donde se convinen todas las palabras de la lista de columnas.

In [11]:
columnas_convinadas = ['genres','overview']
df['texto_combinado'] = df[columnas_convinadas].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)

In [12]:
celda_completa = df.loc[0, 'texto_combinado']
print(celda_completa)

Animation, Comedy, Family Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.


aplico la tokenizacion, elimino los stop_words y lemmatizo el texto.

In [13]:
df['texto_conbinado_tokenizado'] = df['texto_combinado'].apply(lambda x: word_tokenize(x))

In [14]:
stop_words = set(stopwords.words('english'))
df['texto_conbinado_sin_stopwords'] = df['texto_conbinado_tokenizado'].apply(lambda x: [word for word in x if word.lower() not in stop_words])

In [15]:
lemmatizer = WordNetLemmatizer()
df['texto_lemmatizado'] = df['texto_conbinado_sin_stopwords'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [16]:
celda_completa = df.loc[0, 'texto_lemmatizado']
print(celda_completa)

['Animation', ',', 'Comedy', ',', 'Family', 'Led', 'Woody', ',', 'Andy', "'s", 'toy', 'live', 'happily', 'room', 'Andy', "'s", 'birthday', 'brings', 'Buzz', 'Lightyear', 'onto', 'scene', '.', 'Afraid', 'losing', 'place', 'Andy', "'s", 'heart', ',', 'Woody', 'plot', 'Buzz', '.', 'circumstance', 'separate', 'Buzz', 'Woody', 'owner', ',', 'duo', 'eventually', 'learns', 'put', 'aside', 'difference', '.']


Se crea el modelo utilizando la tecnica de Bolsa de Palabras y la similitud del coseno para encontrar similitudes entre los textos lematizados. Se guarda el modelo en un archivo utilizando joblib.

In [39]:
def crear_modelo_recomendacion(df):
    texto_lemmatizado = [' '.join(tokens) for tokens in df['texto_lemmatizado']]

    vectorizer = CountVectorizer()
    matriz_tdm = vectorizer.fit_transform(texto_lemmatizado).astype(np.float32)

    vocabulario = vectorizer.get_feature_names_out()

    similarity_matrix = cosine_similarity(matriz_tdm)

    modelo_recomendacion = {
        'df': df,
        'vectorizer': vectorizer,
        'similarity_matrix': similarity_matrix
    }

    return modelo_recomendacion


modelo_recomendacion = crear_modelo_recomendacion(df)

joblib.dump(modelo_recomendacion, 'modelo_joblib.joblib')


['modelo_joblib.joblib']

In [24]:
df = df[['title', 'texto_lemmatizado']]

In [25]:
df2 = pd.read_csv(r'df_modificado.csv')

In [26]:
df = pd.merge(df, df2, on='title')
df = df.head(3000)

In [38]:
df.to_csv(r'df_modificado.csv', index=False)

El archivo con el modelo entrenado pesa 7GB por lo que procedo a reducir el DataFrame para cumplir con los requisitos de almacenamiento en github (2gb)

In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 3000 non-null   object 
 1   texto_lemmatizado     3000 non-null   object 
 2   genres                2791 non-null   object 
 3   overview              2999 non-null   object 
 4   popularity            3000 non-null   float64
 5   production_companies  2416 non-null   object 
 6   runtime               3000 non-null   float64
 7   tagline               1654 non-null   object 
 8   vote_average          3000 non-null   float64
 9   vote_count            3000 non-null   float64
 10  cast                  2929 non-null   object 
 11  crew                  2992 non-null   object 
dtypes: float64(4), object(8)
memory usage: 281.4+ KB


In [19]:
df = df.head(3000)

In [34]:
df = df.drop(['texto_lemmatizado_y'], axis=1)

In [36]:
df = df.rename(columns={'texto_lemmatizado_x': 'texto_lemmatizado'})

In [22]:
df = df.drop(['texto_combinado', 'texto_conbinado_tokenizado', 'texto_conbinado_sin_stopwords'], axis=1)

In [ ]:
df.to_csv(r'df_modificado.csv', index=False)